Import Libraries

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\phili\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\phili\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\phili\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\phili\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\phili\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np

import pickle as pk

import warnings
warnings.filterwarnings("ignore")


from bs4 import BeautifulSoup
import unicodedata
import re

import nltk
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
import spacy
from nltk.corpus import stopwords


from nltk.corpus import wordnet
from nltk import pos_tag
from nltk import ne_chunk

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from wordcloud import WordCloud

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\phili\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Load data

In [3]:
train_path = "data/train.csv/train.csv"
test_path = "data/test.csv/test.csv"
test_labels_path = "data/test_labels.csv/test_labels.csv"

df = pd.read_csv(train_path)
df.drop('id', inplace=True, axis=1) #dropping id column
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df.dtypes

comment_text     object
toxic             int64
severe_toxic      int64
obscene           int64
threat            int64
insult            int64
identity_hate     int64
dtype: object

In [5]:
df['comment_text'] = df['comment_text'].astype(str)

Preprocessing

- Lower case
- removing accented characs
- removing punctuation
- removing irrelevant characters
- removing extra whitespaces


In [6]:
def rem_accented(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
def rem_puntucation(text):
    pat = r'[^a-zA-Z0-9;\"\'\s]'
    return re.sub(pat, '',text)
def get_long(text):
    text = re.sub(r"can't", "can not",text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'s", " is ", text)
    return text
def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!"?/:;\"\'\s]' 
    return re.sub(pat, '', text)
    
# stopwords
stops = stopwords.words("english") 
delete = ['no',
'nor',
'not',
'don',
"don't",
'ain',
'aren',
"aren't",
'couldn',
"couldn't",
'didn',
"didn't",
'doesn',
"doesn't",
'hadn',
"hadn't",
'hasn',
"hasn't",
'haven',
"haven't",
'isn',
"isn't",
'ma',
'mightn',
"mightn't",
'mustn',
"mustn't",
'needn',
"needn't",
'shan',
"shan't",
'shouldn',
"shouldn't",
'wasn',
"wasn't",
'weren',
"weren't",
'won',
"won't",
'wouldn',
"wouldn't"]
new = [stop for stop in stops if stop not in delete]

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

# def get_lemmatization(text):
#     return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

def get_lemmatization(text):
    # Tokenize: Split the sentence into words
    word_list = nltk.word_tokenize(text)
    # Lemmatize list of words and join
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

In [7]:
df['clean_comment'] = df['comment_text'].str.lower()
df['clean_comment'] = df['clean_comment'].apply(rem_puntucation)
df['clean_comment'] = df['clean_comment'].apply(get_long)
df['clean_comment'] = df['clean_comment'].apply(remove_special_characters)
df['clean_comment'] = df['clean_comment'].apply(lambda x: ' '.join([item for item in x.split() if item not in new]))
df['clean_comment'] = df['clean_comment'].apply(rem_accented)
df['clean_comment'] = df['clean_comment'].apply(get_lemmatization)
df['clean_comment'] = [n.replace('"','') for n in df['clean_comment']]
# df['clean_comment'] = df['clean_comment'].apply(lambda x: ' '.join([item for item in x.split() if item not in stops]))

In [8]:
df

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_comment
0,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0,explanation edits made username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww match background colour seemingly stuck ...
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man really not trying edit war guy constan...
3,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0,`` not make real suggestion improvement wonder...
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,sir hero chance remember page
...,...,...,...,...,...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,`` and second time asking view completely cont...
159567,You should be ashamed of yourself \r\n\r\nThat...,0,0,0,0,0,0,ashamed horrible thing put talk page 128611993
159568,"Spitzer \r\n\r\nUmm, theres no actual article ...",0,0,0,0,0,0,spitzer umm there no actual article prostituti...
159569,And it looks like it was actually you who put ...,0,0,0,0,0,0,look like actually put speedy first version de...


In [9]:
df.columns

Index(['comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate', 'clean_comment'],
      dtype='object')

In [10]:
from collections import Counter

In [11]:
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


In [12]:
counter = counter_word(df.clean_comment)

In [13]:
counter

Counter({'explanation': 2052,
         'edits': 9884,
         'made': 9569,
         'username': 1784,
         'hardcore': 146,
         'metallica': 38,
         'fan': 1286,
         'reverted': 3876,
         'were': 453,
         "n't": 51788,
         'vandalism': 5984,
         'closure': 96,
         'gas': 348,
         'voted': 297,
         'new': 10462,
         'york': 918,
         'doll': 35,
         'fac': 293,
         'please': 29601,
         'do': 23108,
         'remove': 5255,
         'template': 3929,
         'talk': 31582,
         'page': 56433,
         'since': 8661,
         'retired': 223,
         'now892053827': 2,
         "d'aww": 1,
         'match': 959,
         'background': 828,
         'colour': 290,
         'seemingly': 163,
         'stuck': 337,
         'thanks': 13760,
         '2151': 8,
         'january': 1146,
         '11': 953,
         '2016': 72,
         'utc': 6781,
         'hey': 2844,
         'man': 2852,
         'really'

In [14]:
counter.most_common(5)

[("''", 236943),
 ('``', 163081),
 ('not', 101855),
 ('article', 72915),
 ('page', 56433)]

In [15]:
# Split dataset into training and test dataset
train_size = int(df.shape[0]*0.8)
train_df = df[:train_size]
val_df = df[train_size:]

# Split text and labels
train_sentences = train_df.clean_comment.to_numpy()
train_labels = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate']].to_numpy() 
val_sentences = val_df.clean_comment.to_numpy()
val_labels = val_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate']].to_numpy() 

In [16]:
num_unique_words = len(counter)

In [17]:
train_sentences.shape,val_sentences.shape

((127656,), (31915,))

In [18]:
#Tokenize
from lib2to3.pgen2.tokenize import tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences)

In [19]:
word_index = tokenizer.word_index

In [20]:
word_index

{"''": 1,
 'not': 2,
 'article': 3,
 'page': 4,
 "n't": 5,
 'wikipedia': 6,
 'talk': 7,
 'would': 8,
 'one': 9,
 'please': 10,
 'no': 11,
 'like': 12,
 'do': 13,
 'see': 14,
 'source': 15,
 'think': 16,
 'also': 17,
 'know': 18,
 'i': 19,
 'time': 20,
 'people': 21,
 'edit': 22,
 'use': 23,
 'make': 24,
 'may': 25,
 'get': 26,
 "'": 27,
 'say': 28,
 'need': 29,
 'thanks': 30,
 'user': 31,
 'even': 32,
 'link': 33,
 'name': 34,
 'could': 35,
 'good': 36,
 'want': 37,
 'information': 38,
 'way': 39,
 'editor': 40,
 'comment': 41,
 'well': 42,
 'go': 43,
 'image': 44,
 'section': 45,
 'deletion': 46,
 'thing': 47,
 'help': 48,
 'u': 49,
 'first': 50,
 'question': 51,
 'fact': 52,
 'new': 53,
 'look': 54,
 'editing': 55,
 'work': 56,
 'discussion': 57,
 'point': 58,
 'edits': 59,
 'thank': 60,
 "'ve": 61,
 'right': 62,
 'made': 63,
 'many': 64,
 'much': 65,
 'take': 66,
 'used': 67,
 'find': 68,
 'really': 69,
 'fuck': 70,
 'deleted': 71,
 'reference': 72,
 'read': 73,
 'since': 74,
 'add'

In [21]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [22]:
df.iloc[10:15]

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_comment
10,"""\r\nFair use rationale for Image:Wonju.jpg\r\...",0,0,0,0,0,0,`` fair use rationale imagewonjujpg thanks upl...
11,bbq \r\n\r\nbe a man and lets discuss it-maybe...,0,0,0,0,0,0,bbq man let discus itmaybe phone
12,Hey... what is it..\r\n@ | talk .\r\nWhat is i...,1,0,0,0,0,0,hey talk exclusive group wp talibanswho good d...
13,Before you start throwing accusations and warn...,0,0,0,0,0,0,start throwing accusation warning let review e...
14,"Oh, and the girl above started her arguments w...",0,0,0,0,0,0,oh girl started argument stuck nose doe n't be...


In [23]:
print(train_sentences[10:15])
print(train_sequences[10:15])

["`` fair use rationale imagewonjujpg thanks uploading imagewonjujpg notice image page specifies image used fair use no explanation rationale use wikipedia article constitutes fair use addition boilerplate fair use template must also write image description page specific explanation rationale using image article consistent fair use please go image description page edit include fair use rationale uploaded fair use medium consider checking specified fair use rationale page find list 'image ' page edited clicking `` '' my contribution '' '' link located top wikipedia page logged selecting `` '' image '' '' dropdown box note fair use image uploaded 4 may 2006 lacking explanation deleted one week uploaded described criterion speedy deletion question please ask medium copyright question page thank talk contribs unspecified source imagewonjujpg thanks uploading imagewonjujpg noticed file description page currently doe n't specify created content copyright status unclear not create file need s

In [24]:
# Pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Max number of words in sequences
max_length = 20

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, val_padded.shape

((127656, 20), (31915, 20))

In [25]:
train_padded[10]

array([  208,    23,   691, 35210,    30,  1156, 35210,   266,    44,
           4,  2380,    44,    67,   208,    23,    11,   449,   691,
          23,     6])

In [27]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

`` fair use rationale imagewonjujpg thanks uploading imagewonjujpg notice image page specifies image used fair use no explanation rationale use wikipedia article constitutes fair use addition boilerplate fair use template must also write image description page specific explanation rationale using image article consistent fair use please go image description page edit include fair use rationale uploaded fair use medium consider checking specified fair use rationale page find list 'image ' page edited clicking `` '' my contribution '' '' link located top wikipedia page logged selecting `` '' image '' '' dropdown box note fair use image uploaded 4 may 2006 lacking explanation deleted one week uploaded described criterion speedy deletion question please ask medium copyright question page thank talk contribs unspecified source imagewonjujpg thanks uploading imagewonjujpg noticed file description page currently doe n't specify created content copyright status unclear not create file need spe

In [28]:
# Check reversing the induces

# flip (key,values) dictionary
reverse_word_index = dict([(idx,word) for (word, idx) in word_index.items()])

In [29]:
reverse_word_index

{1: "''",
 2: 'not',
 3: 'article',
 4: 'page',
 5: "n't",
 6: 'wikipedia',
 7: 'talk',
 8: 'would',
 9: 'one',
 10: 'please',
 11: 'no',
 12: 'like',
 13: 'do',
 14: 'see',
 15: 'source',
 16: 'think',
 17: 'also',
 18: 'know',
 19: 'i',
 20: 'time',
 21: 'people',
 22: 'edit',
 23: 'use',
 24: 'make',
 25: 'may',
 26: 'get',
 27: "'",
 28: 'say',
 29: 'need',
 30: 'thanks',
 31: 'user',
 32: 'even',
 33: 'link',
 34: 'name',
 35: 'could',
 36: 'good',
 37: 'want',
 38: 'information',
 39: 'way',
 40: 'editor',
 41: 'comment',
 42: 'well',
 43: 'go',
 44: 'image',
 45: 'section',
 46: 'deletion',
 47: 'thing',
 48: 'help',
 49: 'u',
 50: 'first',
 51: 'question',
 52: 'fact',
 53: 'new',
 54: 'look',
 55: 'editing',
 56: 'work',
 57: 'discussion',
 58: 'point',
 59: 'edits',
 60: 'thank',
 61: "'ve",
 62: 'right',
 63: 'made',
 64: 'many',
 65: 'much',
 66: 'take',
 67: 'used',
 68: 'find',
 69: 'really',
 70: 'fuck',
 71: 'deleted',
 72: 'reference',
 73: 'read',
 74: 'since',
 75: '

In [31]:
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [33]:
decode_text = decode(train_sequences[10])

print(train_sequences[10])
print(decode_text)

[208, 23, 691, 35210, 30, 1156, 35210, 266, 44, 4, 2380, 44, 67, 208, 23, 11, 449, 691, 23, 6, 3, 2102, 208, 23, 420, 6613, 208, 23, 227, 155, 17, 240, 44, 478, 4, 376, 449, 691, 128, 44, 3, 1689, 208, 23, 10, 43, 44, 478, 4, 22, 323, 208, 23, 691, 540, 208, 23, 250, 269, 1214, 1680, 208, 23, 691, 4, 68, 77, 2018, 27, 4, 453, 1339, 1, 1917, 170, 1, 1, 33, 1505, 288, 6, 4, 1464, 2645, 1, 44, 1, 1, 2925, 708, 95, 208, 23, 44, 540, 407, 25, 458, 2153, 449, 71, 9, 360, 540, 776, 220, 166, 46, 51, 10, 126, 250, 146, 51, 4, 60, 7, 422, 5251, 15, 35210, 30, 1156, 35210, 485, 425, 478, 4, 393, 116, 5, 1742, 222, 84, 146, 514, 1809, 2, 339, 425, 29, 1742, 1589, 146, 2974, 194, 33, 194, 402, 627, 5526, 194, 152, 23, 84, 676, 1475, 38, 115, 146, 1635, 177, 194, 1937, 146, 17, 2801, 42, 214, 15, 10, 75, 620, 146, 2873, 106, 425, 116, 5, 9, 147, 3888, 245, 2646, 608, 106, 67, 679, 1789, 131, 250, 488, 220, 2995, 23, 23, 106, 9, 106, 296, 1817, 146, 5490, 23, 14, 1817, 146, 106, 412, 77, 146, 106, 2

Create LSTM model

In [39]:
from tensorflow.keras import layers
from tensorflow import keras
model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words,32,input_length=max_length))
model.add(layers.LSTM(64,dropout=0.1))
model.add(layers.Dense(6,activation="sigmoid"))

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 32)            7600128   
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 7,625,350
Trainable params: 7,625,350
Non-trainable params: 0
_________________________________________________________________


In [41]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr= 0.001)
metrics = ['accuracy']

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [42]:
model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=2)

Epoch 1/20
3990/3990 - 297s - loss: 0.0786 - accuracy: 0.9851 - val_loss: 0.0558 - val_accuracy: 0.9940 - 297s/epoch - 74ms/step
Epoch 2/20
3990/3990 - 290s - loss: 0.0480 - accuracy: 0.9911 - val_loss: 0.0540 - val_accuracy: 0.9941 - 290s/epoch - 73ms/step
Epoch 3/20
3990/3990 - 290s - loss: 0.0395 - accuracy: 0.9773 - val_loss: 0.0561 - val_accuracy: 0.9873 - 290s/epoch - 73ms/step
Epoch 4/20
3990/3990 - 289s - loss: 0.0333 - accuracy: 0.9574 - val_loss: 0.0620 - val_accuracy: 0.7257 - 289s/epoch - 72ms/step
Epoch 5/20
3990/3990 - 292s - loss: 0.0282 - accuracy: 0.8784 - val_loss: 0.0663 - val_accuracy: 0.6949 - 292s/epoch - 73ms/step
Epoch 6/20
3990/3990 - 292s - loss: 0.0235 - accuracy: 0.8713 - val_loss: 0.0709 - val_accuracy: 0.8357 - 292s/epoch - 73ms/step
Epoch 7/20
3990/3990 - 295s - loss: 0.0194 - accuracy: 0.8625 - val_loss: 0.0799 - val_accuracy: 0.9565 - 295s/epoch - 74ms/step
Epoch 8/20
3990/3990 - 290s - loss: 0.0160 - accuracy: 0.8513 - val_loss: 0.0877 - val_accuracy: 

In [43]:
predictions = model.predict(train_padded)

In [48]:
predictions 

array([[4.6831383e-07, 1.0460631e-09, 6.2898160e-08, 7.5215194e-09,
        6.1366605e-09, 3.4915274e-08],
       [3.2436429e-07, 1.4045770e-09, 3.9731692e-08, 6.3613008e-08,
        1.9281008e-05, 5.5584933e-06],
       [2.5811344e-02, 2.7802341e-06, 2.4860799e-03, 2.5800422e-05,
        1.2832840e-05, 9.5290115e-06],
       ...,
       [1.6929062e-07, 3.5047648e-08, 1.4055728e-07, 6.6246838e-07,
        1.3290160e-08, 9.9592739e-07],
       [9.9977678e-01, 8.4266067e-04, 1.9981563e-03, 3.9005280e-04,
        9.3631923e-01, 5.0599211e-01],
       [3.6485520e-08, 1.5898065e-10, 1.3191881e-06, 2.3954698e-07,
        1.8265139e-08, 2.8044738e-07]], dtype=float32)

In [50]:
print(train_sentences[10:20])

["`` fair use rationale imagewonjujpg thanks uploading imagewonjujpg notice image page specifies image used fair use no explanation rationale use wikipedia article constitutes fair use addition boilerplate fair use template must also write image description page specific explanation rationale using image article consistent fair use please go image description page edit include fair use rationale uploaded fair use medium consider checking specified fair use rationale page find list 'image ' page edited clicking `` '' my contribution '' '' link located top wikipedia page logged selecting `` '' image '' '' dropdown box note fair use image uploaded 4 may 2006 lacking explanation deleted one week uploaded described criterion speedy deletion question please ask medium copyright question page thank talk contribs unspecified source imagewonjujpg thanks uploading imagewonjujpg noticed file description page currently doe n't specify created content copyright status unclear not create file need s

In [51]:
print(train_labels[10:20])

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [52]:
print(predictions[10:20])

[[2.4693364e-08 3.6605233e-10 3.1585827e-08 5.7443696e-08 3.4045072e-10
  3.2781147e-09]
 [2.5696673e-07 1.3203741e-09 3.5317864e-09 4.8989712e-08 7.1108612e-09
  1.2409784e-08]
 [9.9981070e-01 4.0196855e-06 1.3040057e-06 2.8723971e-08 4.5868310e-06
  1.5698268e-05]
 [1.4719393e-07 1.8243055e-09 4.0054005e-08 4.9573142e-08 5.4199654e-09
  2.2744688e-08]
 [2.3815794e-05 4.4213838e-10 6.8168123e-07 3.6003616e-08 4.6895498e-06
  5.6302527e-09]
 [1.9624311e-07 2.2020399e-09 2.5996513e-08 8.2783792e-08 1.1891338e-08
  3.0961019e-08]
 [9.9630439e-01 2.1502535e-06 1.5524030e-04 6.0069496e-08 2.6929472e-05
  2.0714173e-08]
 [1.5809655e-07 3.5398948e-08 9.8723598e-09 2.9709602e-06 2.1354680e-08
  5.5097846e-08]
 [3.1688771e-08 6.9013108e-11 2.6592744e-09 1.6245217e-09 7.4992297e-09
  8.1641227e-10]
 [9.3583130e-06 4.5047670e-08 2.7466463e-06 2.3750913e-06 8.1554155e-08
  3.8180400e-07]]


In [54]:
train_df[10:20]

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_comment
10,"""\r\nFair use rationale for Image:Wonju.jpg\r\...",0,0,0,0,0,0,`` fair use rationale imagewonjujpg thanks upl...
11,bbq \r\n\r\nbe a man and lets discuss it-maybe...,0,0,0,0,0,0,bbq man let discus itmaybe phone
12,Hey... what is it..\r\n@ | talk .\r\nWhat is i...,1,0,0,0,0,0,hey talk exclusive group wp talibanswho good d...
13,Before you start throwing accusations and warn...,0,0,0,0,0,0,start throwing accusation warning let review e...
14,"Oh, and the girl above started her arguments w...",0,0,0,0,0,0,oh girl started argument stuck nose doe n't be...
15,"""\r\n\r\nJuelz Santanas Age\r\n\r\nIn 2002, Ju...",0,0,0,0,0,0,`` juelz santanas age 2002 juelz santana 18 ye...
16,"Bye! \r\n\r\nDon't look, come or think of comm...",1,0,0,0,0,0,bye do n't look come think comming back tosser
17,REDIRECT Talk:Voydan Pop Georgiev- Chernodrinski,0,0,0,0,0,0,redirect talkvoydan pop georgiev chernodrinski
18,The Mitsurugi point made no sense - why not ar...,0,0,0,0,0,0,mitsurugi point made no sense not argue includ...
19,Don't mean to bother you \r\n\r\nI see that yo...,0,0,0,0,0,0,do n't mean bother see writing something regar...
